
# Export a function as Modelica model


``otfmi.FunctionExporter`` enables to export OpenTURNS functions as Modelica model.
The main interest is to use OpenTURNS metamodels in a simulation environment.

Currently, the inclusion of a metamodel in
[OpenModelica GUI](https://openmodelica.org/?id=78:omconnectioneditoromedit&catid=10:main-category)
has been performed once (see
[this paper](https://www.researchgate.net/publication/354810878_Analysis_and_reduction_of_models_using_Persalys)).

------------

## Model generation

First, we create the OpenTURNS function to export as Modelica model.



In [ ]:
import openturns as ot
import otfmi
import tempfile
from os.path import join


func = ot.SymbolicFunction("x", "exp(x)")
inputPoint = [2.0]
print(func(inputPoint))

We create the model constructor and the folder in which save the model:



In [ ]:
fmuExporter = otfmi.FunctionExporter(func)

model_path = join(tempfile.mkdtemp(), "myExponential.mo")
print(model_path)

We create the FunctionExporter instance and export the exponential function.
We specify ``gui=True`` to use the model in a Modelica GUI in connection
with other components.



In [ ]:
modelExporter = otfmi.FunctionExporter(func)
modelExporter.export_model(model_path, gui=True)

Simple as it looks, this function actually does the following:

- write a C-wrapper for the OpenTURNS function,
- write a Modelica model calling the C-wrapper as [External](shorturl.at/fhCU2) function.

<div class="alert alert-info"><h4>Note</h4><p>The export requires [CMake](https://cmake.org/), a C compiler, and the
  OpenModelica compiler [OMC](https://www.openmodelica.org/?id=51:open-modelica-compiler-omccecatid=10:main-category).</p></div>

----------------------

## Model validation

We import this model in OpenModelica GUI. We can check the Modelica code:

<img src="file://_static/modelica_gui_code.png" scale="80 %" alt="alternate text" align="center">

<div class="alert alert-info"><h4>Note</h4><p>The path to the C-wrapper is hard-coded in the model.</p></div>

We can also check the connectors position:

<img src="file://_static/modelica_gui_interface.png" scale="60 %" alt="alternate text" align="center">

We connect the wrapper to an input sine signal
(*Modelica.Blocks.Sources.Sine*) and to an output block
(*Modelica.Blocks.Interfaces.RealOutput*):

<img src="file://_static/TestMyExponential.png" scale="100 %" alt="alternate text" align="center">

We simulate the model on 1 second, with 50 time steps. We can verify that `y`
output corresponds to the exponential of the sine signal.

<img src="file://_static/show_output.png" scale="80 %" alt="alternate text" align="center">

<div class="alert alert-danger"><h4>Warning</h4><p>⚠️ Compared to native Modelica functions, the included OpenTURNS function is
   slow. In this case, 11 seconds of simulation were required for 50 time
   steps (i.e. 50 function calls).</p></div>

Note that faster export modes are available with the "mode" keyword, depending on your setup.

